<a href="https://colab.research.google.com/github/liverms/visual/blob/master/visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install plotly
!pip install datetime

     |████████████████████████████████| 61kB 2.0MB/s 
     |████████████████████████████████| 174kB 8.1MB/s 


In [2]:
from google.colab import drive
import pandas as pd
import datetime as dt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Below are functions, each function corresponds to a column in the big csv file for the proactive disclosure of contracts.  These functions lean up the columns.

In [0]:
def original_value(df):
    col = 'original_value'
    df[col].fillna(0, inplace=True)
    fix = [
          '$',
          ','
    ]
    for error in fix:
        df[col] = df[col].str.replace(error, '')
    
    df[col] = df[col].str.replace('NA', '0')
    df[col] = df[col].astype(float)
    return df

In [0]:
def amendment_value(df):
    col = 'amendment_value'
    df[col].fillna(0, inplace=True)
    fix = [
          '$',
          ','
    ]
    for error in fix:
        df[col] = df[col].str.replace(error, '')

    df[col] = df[col].str.replace('NA', '0')
    df[col] = df[col].astype(float)
    return df

In [0]:
def document_type_code(df):
    col = 'document_type_code'
    c_type = ['c', 'C ', 'C']
    for error in c_type:
        df[col] = df[col].replace(error, 'Contract')

    fix_doc_type = ['A']
    for error in fix_doc_type:
        df[col] = df[col].replace(error, 'Amendment')

    doc_type = ['Contract', 'Amendment', 'SOSA']
    df = df[df[col].isin(doc_type)]
    return df

In [0]:
def contract_date(df):
  col = 'contract_date'
  
  df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
  df= df[df[col] >= '2004-01-01']
  return df

For this function, there is an if statement checking if there is a column for 'commodity_codes'.  This is because for many entries there is no 'commodity_type_code' given, so this functions determines the commodity_type_code from the commodity_code.  In order to make use of this both columns have to be passed to the load data function.

In [0]:
def commodity_type_code(df):
    col = 'commodity_type_code'
    # Change coding for Goods to string Goods
    fix_goods = [
        'g',
        'GOODS',
        'G: Goods',
        'Good',
        'Goods',
        'G'
    ]
    for error in fix_goods:
        df[col] = df[col].str.replace(error, 'G')

    #change variations of services to string Services
    fix_services = [
        'S: Service',
        'SERVICES',
        'Service',
        's',
        'S'
    ]
    for error in fix_services:
        df[col] = df[col].str.replace(error, 'S')

    # change coding of construction to string Construction
    fix_construction = ['c', 'C']
    for error in fix_construction:
        df[col] = df[col].str.replace(error, 'C')

    commodity_codes = ['G', 'S', 'C']

    if 'commodity_code' in df.columns:
        services = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'R', 'S', 'T', 'U', 'V', 'W',
                          'X',
                          'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'r', 's', 't', 'u', 'v', 'w',
                          'x']
        goods = ['N', 'n', '1', '2', '3', '4', '5', '6', '7', '8', '9']

        construction = ['51']

        for k in services:
            df.loc[(df['commodity_code'].str.slice(0, 1) == k) & (df[col] != 'S'), col] = 'S'
        for k in goods:
            df.loc[(df['commodity_code'].str.slice(0, 1) == k) & (df[col] != 'G'), col] = 'G'
        for k in construction:
            df.loc[(df['commodity_code'].str.slice(0, 2) == k) & (df[col] != 'C'), col] = 'C'
    else:
      pass

    df = df[df[col].isin(commodity_codes)]

    return df

In [0]:
def solicitation_procedure_code(df):
  col = 'solicitation_procedure_code'
  
  fix = [
      'AC:  Advance Contract Award Notice',
      'AC          '
  ]
  for error in fix:
    df[col] = df[col].replace(error, 'AC')

  fix = [
      'TC:  Traditional Competitive',
      'tc',
      'TC          '
  ]
  for error in fix:
    df[col] = df[col].replace(error, 'TC')
  
  fix = [
         'TN:  Traditional Non-Competitive',
         'TN          ',
         'TN     ',
         'tn',
         'Tn'
  ]
  for error in fix:
    df[col] = df[col].replace(error, 'TN')

  sol_types = ['AC', 'OB', 'TN', 'TC']
  df = df[df[col].isin(sol_types)]
  return df

Load data loads the csv file for the proactive disclosure of contracts.  If no arguments are given then it will load these four columns: 'original_value', 'contract_date', 'document_type_code' and 'amendment_value'.  If you want to load more columns then pass the column name in quotations as an argument.  For instance to load with 'commodity_type_code' write the function as load_data('commodity_type_code').

In [0]:
def load_data(*args):
  usecols=[
      'original_value',
      'contract_date',
      'document_type_code',
      'amendment_value'
  ]

  dtype={
      'original_value': str,
      'contract_date': str,
      'document_type_code':str,
      'amendment_value': str
  }
  if args is None:
    pass
  else:
    for col in args:
      usecols.append(col)
      dtype[col] = str
  
  df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/contracts.csv',
                    usecols=usecols,
                    dtype=dtype)
  for col in usecols:
      df[col] = df[col].str.strip()
  
  df = document_type_code(df)

  df.loc[(df['document_type_code'] == 'Amendment'), 'original_value'] = '0'
  df.loc[(df['document_type_code'] == 'Contract'), 'amendment_value'] = '0'
  df = original_value(df)
  df = contract_date(df)
  df = amendment_value(df)
  df['total'] = df['original_value'] + df['amendment_value']  

  return df

Below are the transformations to get the data for the graphs.

In [0]:
def num_val_percent(df, group1, group2, agg):
  df1 = df.groupby([group1, pd.Grouper(freq='Y')]).agg({'total': agg})
  df2 = df1.groupby([group2]).agg({'total': 'sum'})
  df3 = df1.div(df2, level=group2)*100
  df3 = df3.merge(df1, how='inner', on=[group1, group2])
  df3 = df3.rename(columns={'total_x': (agg +' percent'), 'total_y': (agg + ' sum')})
  return df3

In [0]:
def num_val_total(df, group1, group2):
  df1 = df.groupby(group1).sum()
  df1[group2] = 'Total'
  df1 = df1.reset_index()
  return df1

In [0]:
def calculate(df, colname):
    # Transform data to the desired subset
    dfc = df.drop(['original_value', 'amendment_value', 'document_type_code'], axis=1)
    dfc = dfc[dfc['contract_date'] >= '2017-01-01']
    dfc['contract_date'] = dfc['contract_date'].dt.to_period('Y')
    dfc = dfc.set_index('contract_date')
    dfc = dfc[dfc['total'] > 10000]

    # Create two groupby objects and divide one by the other to get percent
    # Do the above for contract value and count, then merge the dfs together.
    dfsum = num_val_percent(dfc, colname, 'contract_date', 'sum')
    dfcount = num_val_percent(dfc, colname, 'contract_date', 'count')
    dfcom = dfsum.merge(dfcount, how='inner', on=[colname, 'contract_date'])
    dfcom = dfcom.reset_index()

    # Get totals
    dftot = num_val_total(dfcom, 'contract_date', colname)
    dftot = dfcom.append(dftot, sort=False)
    dftot = dftot.round(0)
    dftot['count percent'] = pd.to_numeric(dftot['count percent'], downcast='integer')
    dftot['sum percent'] = pd.to_numeric(dftot['sum percent'], downcast='integer')
    return dftot

In [0]:
def solicitation_procedure_code(df):
  col = 'solicitation_procedure_code'
  
  fix = [
      'AC:  Advance Contract Award Notice',
      'AC          '
  ]
  for error in fix:
    df[col] = df[col].replace(error, 'AC')

  fix = [
      'TC:  Traditional Competitive',
      'tc',
      'TC          '
  ]
  for error in fix:
    df[col] = df[col].replace(error, 'TC')
  
  fix = [
         'TN:  Traditional Non-Competitive',
         'TN          ',
         'TN     ',
         'tn',
         'Tn'
  ]
  for error in fix:
    df[col] = df[col].replace(error, 'TN')

  sol_types = ['AC', 'OB', 'TN', 'TC']
  df = df[df[col].isin(sol_types)]
  return df

In [0]:
def fix_commodities(dftot):
  # Make presentable
  dftot.loc[(dftot['commodity_type_code'] == 'C'), 'commodity_type_code'] = 'Construction'
  dftot.loc[(dftot['commodity_type_code'] == 'G'), 'commodity_type_code'] = 'Goods'
  dftot.loc[(dftot['commodity_type_code'] == 'S'), 'commodity_type_code'] = 'Services'
  dftot['contract_date'] = dftot['contract_date'].dt.strftime('%Y')
  dftot = dftot.rename(columns={'commodity_type_code': 'Commodity Type', 'contract_date': 'Year', 'sum percent': 'Value (%)', 'sum sum': 'Value ($)', 'count percent': 'Number of Contracts (%)', 'count sum' : 'Number of Contracts (#)'}) 
  return dftot

In [29]:
# Load data and clean extra columns
#This is for all contracts for commodity type codes.
dft = load_data('commodity_type_code', 'commodity_code')
dft = commodity_type_code(dft)
dft = dft.drop('commodity_code', axis=1)
dft = calculate(dft, 'commodity_type_code')
dft = fix_commodities(dft)
dft

,Commodity Type,Year,Value (%),Value ($),Number of Contracts (%),Number of Contracts (#)
0,Construction,2017,19,2.716255e+09,10,7080
1,Construction,2018,12,1.949687e+09,10,7886
2,Construction,2019,9,1.103298e+09,8,3992
3,Goods,2017,23,3.279002e+09,35,25008
4,Goods,2018,28,4.378841e+09,38,28822
5,Goods,2019,41,5.113311e+09,39,19118
6,Services,2017,57,8.000435e+09,55,39349
7,Services,2018,60,9.340878e+09,52,39528
8,Services,2019,51,6.365949e+09,53,25794
0,Total,2017,100,1.399569e+10,100,71437


In [31]:
# Load data and clean extra columns
# This is contracts under $25k for commodity type codes 
dfu = load_data('commodity_type_code', 'commodity_code')
dfu = commodity_type_code(dfu)
dfu = dfu[dfu['total'] < 25000]
dfu = dfu.drop('commodity_code', axis=1)
dfu = calculate(dfu, 'commodity_type_code')
dfu = fix_commodities(dfu)
dfu

,Commodity Type,Year,Value (%),Value ($),Number of Contracts (%),Number of Contracts (#)
0,Construction,2017,9,51453142.0,9,3137
1,Construction,2018,9,60098459.0,9,3648
2,Construction,2019,8,34121044.0,8,2045
3,Goods,2017,37,221693227.0,38,13378
4,Goods,2018,40,258856740.0,40,15484
5,Goods,2019,40,178708258.0,41,10716
6,Services,2017,54,323486888.0,54,19037
7,Services,2018,51,332069851.0,50,19355
8,Services,2019,52,229202026.0,51,13245
0,Total,2017,100,596633257.0,100,35552


In [33]:
# Load data and clean extra columns
# This is contracts over $25k for commodity type codes
dfo = load_data('commodity_type_code', 'commodity_code')
dfo = commodity_type_code(dfo)
dfo = dfo[dfo['total'] > 25000]
dfo = dfo.drop('commodity_code', axis=1)

dfo = calculate(dfo, 'commodity_type_code')
dfo = fix_commodities(dfo)
dfo

,Commodity Type,Year,Value (%),Value ($),Number of Contracts (%),Number of Contracts (#)
0,Construction,2017,20,2.664702e+09,11,3939
1,Construction,2018,13,1.889513e+09,11,4235
2,Construction,2019,9,1.068977e+09,9,1939
3,Goods,2017,23,3.056934e+09,33,11615
4,Goods,2018,27,4.119584e+09,35,13322
5,Goods,2019,41,4.934353e+09,37,8392
6,Services,2017,57,7.673323e+09,56,20167
7,Services,2018,60,9.005708e+09,53,20049
8,Services,2019,51,6.133097e+09,55,12403
0,Total,2017,100,1.339496e+10,100,35721


In [0]:
def fix_solicitation_procedures(df):
  df.loc[df['solicitation_procedure_code'] == 'AC', 'solicitation_procedure_code'] = 'Advanced Contract Award Notice'
  df.loc[df['solicitation_procedure_code'] == 'OB', 'solicitation_procedure_code'] = 'Open Bidding'
  df.loc[df['solicitation_procedure_code'] == 'TC', 'solicitation_procedure_code'] = 'Traditional Competitive'
  df.loc[df['solicitation_procedure_code'] == 'TN', 'solicitation_procedure_code'] = 'Traditional Non-Competitive'

  df['contract_date'] = df['contract_date'].dt.strftime('%Y')
  df = df.rename(columns={'solicitation_procedure_code': 'Solicitation Procedure', 'contract_date': 'Year', 'sum percent': 'Value (%)', 'sum sum': 'Value ($)', 'count percent': 'Number of Contracts (%)', 'count sum' : 'Number of Contracts (#)'})
  return df

In [50]:
# Load data and clean extra columns
# All contracts for solicitation procedure
dfs = load_data('solicitation_procedure_code')
dfs = solicitation_procedure_code(dfs)

dfs = calculate(dfs, 'solicitation_procedure_code')
dfs = fix_solicitation_procedures(dfs)
dfs

,Solicitation Procedure,Year,Value (%),Value ($),Number of Contracts (%),Number of Contracts (#)
0,Advanced Contract Award Notice,2017,2,3.349566e+08,1,713
1,Advanced Contract Award Notice,2018,7,1.051864e+09,1,769
2,Advanced Contract Award Notice,2019,15,1.857701e+09,1,459
3,Open Bidding,2017,35,4.823415e+09,18,13117
4,Open Bidding,2018,27,4.264271e+09,18,13357
5,Open Bidding,2019,20,2.569799e+09,17,8096
6,Traditional Competitive,2017,48,6.760117e+09,58,40963
7,Traditional Competitive,2018,56,8.681082e+09,58,44194
8,Traditional Competitive,2019,34,4.339690e+09,54,26526
9,Traditional Non-Competitive,2017,15,2.031992e+09,23,16149


In [68]:
# Load data and clean extra columns
# Contracts under $25k for solicitation procedure
dfsu = load_data('solicitation_procedure_code')
dfsu = solicitation_procedure_code(dfsu)
dfsu = dfsu[dfsu['total'] < 25000]
dfsu = calculate(dfsu, 'solicitation_procedure_code')
dfsu = fix_solicitation_procedures(dfsu)
dfsu

,Solicitation Procedure,Year,Value (%),Value ($),Number of Contracts (%),Number of Contracts (#)
0,Advanced Contract Award Notice,2017,0,2784316.0,0,165
1,Advanced Contract Award Notice,2018,0,2754107.0,0,160
2,Advanced Contract Award Notice,2019,0,1246233.0,0,74
3,Open Bidding,2017,11,67850691.0,12,4162
4,Open Bidding,2018,11,72740000.0,12,4426
5,Open Bidding,2019,11,48624553.0,11,2978
6,Traditional Competitive,2017,53,316116873.0,55,19250
7,Traditional Competitive,2018,53,343065897.0,54,20666
8,Traditional Competitive,2019,47,208532477.0,49,12631
9,Traditional Non-Competitive,2017,35,205473905.0,33,11711


In [67]:
# Load data and clean extra columns
# Contracts over $25k for solicitation procedure
dfso = load_data('solicitation_procedure_code')
dfso = solicitation_procedure_code(dfso)
dfso = dfso[dfso['total'] > 25000]
dfso = calculate(dfso, 'solicitation_procedure_code')
dfso = fix_solicitation_procedures(dfso)
dfso

,Solicitation Procedure,Year,Value (%),Value ($),Number of Contracts (%),Number of Contracts (#)
0,Advanced Contract Award Notice,2017,2,3.321223e+08,2,546
1,Advanced Contract Award Notice,2018,7,1.048985e+09,2,604
2,Advanced Contract Award Notice,2019,15,1.856455e+09,2,385
3,Open Bidding,2017,36,4.755364e+09,25,8947
4,Open Bidding,2018,28,4.191381e+09,24,8925
5,Open Bidding,2019,21,2.521074e+09,22,5114
6,Traditional Competitive,2017,48,6.442725e+09,61,21662
7,Traditional Competitive,2018,56,8.337141e+09,63,23493
8,Traditional Competitive,2019,34,4.130507e+09,61,13869
9,Traditional Non-Competitive,2017,14,1.823943e+09,12,4335


In [84]:
# Figure for solicitation procedures

fig = make_subplots(
    rows = 2,
    cols=2,
    shared_xaxes=False,
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
           [{'type': 'table'}, {'type': 'table'}]]
)
#total

def sol_graph(df, name, colnum, vis):
  if name == 'Advanced Contract Award Notice':
    color = 'rgb(0,81,114)'
  elif name == 'Open Bidding':
    color = 'rgb(99,206,202)'
  elif name == 'Traditional Competitive':
    color = 'rgb(204,220,0)'
  elif name == 'Total':
    color = 'rgb(205,32,44)'
  elif name == 'Traditional Non-Competitive':
    color = 'rgb(55,66,74)'
  
  if colnum == 1:
    showlegend = True
    colname = 'Value ($)'
    hover = 'Value (%)'
  elif colnum == 2:
    showlegend = False
    colname = 'Number of Contracts (#)'
    hover = 'Number of Contracts (%)'


  return   fig.add_trace(
              go.Scatter(
                  y = df[df['Solicitation Procedure'] == name][colname],
                  x = df[df['Solicitation Procedure'] == name]['Year'],
                  hovertext = [str(p) + '%' for p in list(df[df['Solicitation Procedure'] == name][hover])],
                  marker=dict(color=color),
                  visible=vis,
                  name = name,
                  showlegend=showlegend
              ),
              row = 1,
              col = colnum
              )


def sol_table(df, colnum):
  if colnum == 1:
    df = df.drop(['Number of Contracts (#)', 'Number of Contracts (%)'], axis=1)
    colname = 'Value ($)'
  elif colnum == 2:
    df = df.drop(['Value ($)', 'Value (%)'], axis=1)
    colname = 'Number of Contracts (#)'

  df[colname] = [f'{k:,}' for k in df[colname]]
  
  return fig.add_trace(
      go.Table(
          header=dict(
              values=[k for k in df.columns]
          ),
          cells = dict(
              values=[df[k].tolist() for k in df[:]]
          )
          ),
          row=2,
          col=colnum
      )

vt = []
vu = []
vo= []
for k in dfs['Solicitation Procedure'].unique():
  sol_graph(dfs, k, 1, True)
  vt.append(True)
  vu.append(False)
  vo.append(False)
for k in dfs['Solicitation Procedure'].unique():
  sol_graph(dfsu, k, 1, False)
  vt.append(False)
  vu.append(True)
  vo.append(False)
for k in dfs['Solicitation Procedure'].unique():
  sol_graph(dfso, k, 1, False)
  vt.append(False)
  vu.append(False)
  vo.append(True)
for k in dfs['Solicitation Procedure'].unique():
  sol_graph(dfs, k, 2, True)
  vt.append(True)
  vu.append(False)
  vo.append(False)
for k in dfs['Solicitation Procedure'].unique():
  sol_graph(dfsu, k, 2, False)
  vt.append(False)
  vu.append(True)
  vo.append(False)
for k in dfs['Solicitation Procedure'].unique():
  sol_graph(dfso, k, 2, False)
  vt.append(False)
  vu.append(False)
  vo.append(True)

sol_table(dfs, 1)
vt.append(True)
sol_table(dfsu, 1)
vu.append(False)
sol_table(dfso, 1)
vo.append(False)

sol_table(dfs, 2)
vt.append(True)
sol_table(dfsu, 2)
vu.append(False)
sol_table(dfso, 2)
vo.append(False)


fig.update_layout(
    template='plotly_white',
    title='Contracts by Solicitation Procedures',
    updatemenus=[
      go.layout.Updatemenu(
          active = 0,
          name = 'Year',
          x = 1,
          y = 1.15,
          showactive=True,
          buttons=list([
                        dict(label='Total',
                             method='update',
                             args=[{'visible': vt}]
                             ),
                        dict(label='Under $25,000',
                             method='update',
                             args=[{'visible': vu}]
                             ),
                        dict(label='Over $25,000',
                             method='update',
                             args=[{'visible': vo}]
                             )
          ]
      ) 
    )
    ],
    annotations=[
                 go.layout.Annotation(
                     text = '<b>Options: </b>',
                     x = 0.92,
                     xref = 'paper',
                     y = 1.12,
                     yref = 'paper',
                     align = 'left',
                     showarrow = False
                 )
    ]
)

fig.update_yaxes(title_text='Value ($)', row=1, col=1)
fig.update_yaxes(title_text='Number of Contracts (#)', row=1, col=2)
fig.update_xaxes(title_text='Year', row=1, col=1, dtick=1)
fig.update_xaxes(title_text='Year', row=1, col=2, dtick=1)

fig.show()

In [87]:
# Figure for commodity types

fig = make_subplots(
    rows = 2,
    cols=2,
    shared_xaxes=False,
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}],
           [{'type': 'table'}, {'type': 'table'}]]
)
#total

def com_graph(df, name, colnum, vis):
  if name == 'Construction':
    color = 'rgb(0,81,114)'
  elif name == 'Services':
    color = 'rgb(99,206,202)'
  elif name == 'Goods':
    color = 'rgb(204,220,0)'
  elif name == 'Total':
    color = 'rgb(205,32,44)'
  
  if colnum == 1:
    showlegend = True
    colname = 'Value ($)'
    hover = 'Value (%)'
  elif colnum == 2:
    showlegend = False
    colname = 'Number of Contracts (#)'
    hover = 'Number of Contracts (%)'


  return   fig.add_trace(
              go.Scatter(
                  y = df[df['Commodity Type'] == name][colname],
                  x = df[df['Commodity Type'] == name]['Year'],
                  hovertext = [str(p) + '%' for p in list(df[df['Commodity Type'] == name][hover])],
                  marker=dict(color=color),
                  visible=vis,
                  name = name,
                  showlegend=showlegend
              ),
              row = 1,
              col = colnum
              )


def com_table(df, colnum):
  if colnum == 1:
    df = df.drop(['Number of Contracts (#)', 'Number of Contracts (%)'], axis=1)
    colname = 'Value ($)'
  elif colnum == 2:
    df = df.drop(['Value ($)', 'Value (%)'], axis=1)
    colname = 'Number of Contracts (#)'

  df[colname] = [f'{k:,}' for k in df[colname]]
  
  return fig.add_trace(
      go.Table(
          header=dict(
              values=[k for k in df.columns]
          ),
          cells = dict(
              values=[df[k].tolist() for k in df[:]]
          )
          ),
          row=2,
          col=colnum
      )

vt = []
vu = []
vo= []
for k in dft['Commodity Type'].unique():
  com_graph(dft, k, 1, True)
  vt.append(True)
  vu.append(False)
  vo.append(False)
for k in dft['Commodity Type'].unique():
  com_graph(dfu, k, 1, False)
  vt.append(False)
  vu.append(True)
  vo.append(False)
for k in dft['Commodity Type'].unique():
  com_graph(dfo, k, 1, False)
  vt.append(False)
  vu.append(False)
  vo.append(True)
for k in dft['Commodity Type'].unique():
  com_graph(dft, k, 2, True)
  vt.append(True)
  vu.append(False)
  vo.append(False)
for k in dft['Commodity Type'].unique():
  com_graph(dfu, k, 2, False)
  vt.append(False)
  vu.append(True)
  vo.append(False)
for k in dft['Commodity Type'].unique():
  com_graph(dfo, k, 2, False)
  vt.append(False)
  vu.append(False)
  vo.append(True)

com_table(dft, 1)
vt.append(True)
com_table(dfu, 1)
vu.append(False)
com_table(dfo, 1)
vo.append(False)

com_table(dfs, 2)
vt.append(True)
com_table(dfu, 2)
vu.append(False)
com_table(dfo, 2)
vo.append(False)


fig.update_layout(
    template='plotly_white',
    title='Contracts by Commodity Type',
    updatemenus=[
      go.layout.Updatemenu(
          active = 0,
          name = 'Year',
          x = 1,
          y = 1.15,
          showactive=True,
          buttons=list([
                        dict(label='Total',
                             method='update',
                             args=[{'visible': vt}]
                             ),
                        dict(label='Under $25,000',
                             method='update',
                             args=[{'visible': vu}]
                             ),
                        dict(label='Over $25,000',
                             method='update',
                             args=[{'visible': vo}]
                             )
          ]
      ) 
    )
    ],
    annotations=[
                 go.layout.Annotation(
                     text = '<b>Options: </b>',
                     x = 0.92,
                     xref = 'paper',
                     y = 1.12,
                     yref = 'paper',
                     align = 'left',
                     showarrow = False
                 )
    ]
)

fig.update_yaxes(title_text='Value ($)', row=1, col=1)
fig.update_yaxes(title_text='Number of Contracts (#)', row=1, col=2)
fig.update_xaxes(title_text='Year', row=1, col=1, dtick=1)
fig.update_xaxes(title_text='Year', row=1, col=2, dtick=1)

fig.show()